In [ ]:
不知道你是否發現身邊聊異步的人越來越多了，比如：FastAPI、Tornado、Sanic、Django 3、aiohttp 等。
聽說異步如何如何牛逼？性能如何吊炸天。。。。但他到底是怎麼一回事呢？
本節要跟大家一起聊聊關於 asyncio 異步的那些事！

[異步編程講義](https://pythonav.com/wiki/detail/6/91/)<br>
[異步編程影片](https://www.bilibili.com/video/BV1dD4y127bD?p=3&spm_id_from=pageDriver)

# 協程

In [ ]:
def func1():
    print(1)
    ...
    print(2)
    
def func2():
    print(3)
    ...
    print(4)
    
func1()
func2()

###  greenlet

In [ ]:
greentlet是一個第三方套件，需要提前安裝 pip3 install greenlet 才能使用。

In [ ]:
from greenlet import greenlet

def func1():
    print(1)        # 第1步：輸出 1
    gr2.switch()    # 第3步：切換到 func2 函數
    print(2)        # 第6步：輸出 2
    gr2.switch()    # 第7步：切換到 func2 函數，從上一次執行的位置繼續向後執行
    
def func2():
    print(3)        # 第4步：輸出 3
    gr1.switch()    # 第5步：切換到 func1 函數，從上一次執行的位置繼續向後執行
    print(4)        # 第8步：輸出 4
    
gr1 = greenlet(func1)
gr2 = greenlet(func2)
gr1.switch() # 第1步：去執行 func1 函數

### yield

In [ ]:
def func1():
    yield 1
    yield from func2()
    yield 2
    
def func2():
    yield 3
    yield 4
    
f1 = func1()
for item in f1:
    print(item)

### asyncio

In [ ]:
需要事先安裝 pip install nest_asyncio

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

@asyncio.coroutine
def func1():
    print(1)
    yield from asyncio.sleep(2)  # 遇到 IO 耗時操作，自動化切換到 tasks 中的其他任務
    print(2)
    
@asyncio.coroutine
def func2():
    print(3)
    yield from asyncio.sleep(2) # 遇到 IO 耗時操作，自動化切換到 tasks 中的其他任務
    print(4)
    
tasks = [
    asyncio.ensure_future( func1() ),
    asyncio.ensure_future( func2() )
]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks));

### async & await

async & await 關鍵字在 Python3.5 版本中正式引入，基於他編寫的協程代碼其實就是 "上一範例" 的加強版，讓代碼可以更加簡便。
Python3.8 之後 @asyncio.coroutine 裝飾器就會被移除，推薦使用 async & awit 關鍵字實現協程代碼。

In [ ]:
import asyncio

async def func1():
    print(1)
    await asyncio.sleep(2)
    print(2)
    
async def func2():
    print(3)
    await asyncio.sleep(2)
    print(4)
    
tasks = [
    asyncio.ensure_future(func1()),
    asyncio.ensure_future(func2())
]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks));

# 協程的意義

### 爬蟲案例

In [ ]:
import requests

def download_image(url):
    print("開始下載:", url)
    # 發送網絡請求，下載圖片
    response = requests.get(url)
    print("下載完成")
    # 圖片保存到本地文件
    file_name = url.rsplit('_')[-1]
    with open(f"mod25/{file_name}", mode='wb') as file_object:
        file_object.write(response.content)
        
if __name__ == '__main__':
    url_list = [
        'https://www3.autoimg.cn/newsdfs/g26/M02/35/A9/120x90_0_autohomecar__ChsEe12AXQ6AOOH_AAFocMs8nzU621.jpg',
        'https://www2.autoimg.cn/newsdfs/g30/M01/3C/E2/120x90_0_autohomecar__ChcCSV2BBICAUntfAADjJFd6800429.jpg',
        'https://www3.autoimg.cn/newsdfs/g26/M0B/3C/65/120x90_0_autohomecar__ChcCP12BFCmAIO83AAGq7vK0sGY193.jpg'
    ]
    
    for item in url_list:
        download_image(item)

---

In [ ]:
方式二：基於協程的異步編程實現

In [ ]:
需先安裝套件 pip install aiohttp

In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import aiohttp
import asyncio

async def fetch(session, url):
    print("發送請求：", url)
    async with session.get(url, verify_ssl=False) as response:
        content = await response.content.read()
        file_name = url.rsplit('_')[-1]
        with open(f"mod25/{file_name}", mode='wb') as file_object:
            file_object.write(content)
            print("下載完成")
            
async def main():
    async with aiohttp.ClientSession() as session:
        url_list = [
            'https://www3.autoimg.cn/newsdfs/g26/M02/35/A9/120x90_0_autohomecar__ChsEe12AXQ6AOOH_AAFocMs8nzU621.jpg',
            'https://www2.autoimg.cn/newsdfs/g30/M01/3C/E2/120x90_0_autohomecar__ChcCSV2BBICAUntfAADjJFd6800429.jpg',
            'https://www3.autoimg.cn/newsdfs/g26/M0B/3C/65/120x90_0_autohomecar__ChcCP12BFCmAIO83AAGq7vK0sGY193.jpg'
        ]
        tasks = [asyncio.create_task(fetch(session, url)) for url in url_list]
        await asyncio.wait(tasks)
        
if __name__ == '__main__':
#     await main()
    asyncio.run(main())
# 在 vscode 跟 jupyter 運行時，這裡需要適當的調整

### 小結

# 異步編程

### 事件循環

In [ ]:
import asyncio

# 去生成或獲取一個事件循環
loop = asyncio.get_event_loop()

# 將任務放到任務列表
loop.run_until_complete(任務)

### 快速上手

In [ ]:
# 定義一個協程函數
async def func():
    pass

# 調用協程函數，返回一個協程對象
result = func()

In [ ]:
import asyncio

async def func():
    print("協程內部代碼")

# 調用協程函數，返回一個協程對象。
result = func()

# 方式一
# 在 jupyter 還需要微調，可直接複製去 vscode 運行 
loop = asyncio.get_event_loop() # 創建一個事件循環
loop.run_until_complete(result) # 將協程當做任務提交到事件循環的任務列表中，協程執行完成之後終止。

In [ ]:
import asyncio

async def func():
    print("協程內部代碼")

# 調用協程函數，返回一個協程對象。
result = func()


# 方式二
# 在 jupyter 還需要微調，把 asyncio.run(result) 換成 await result，也可直接複製去 vscode 運行 
# 本質上方式一是一樣的，內部先 創建事件循環 然後執行 run_until_complete，一個簡便的寫法。
# asyncio.run 函數在 Python 3.7 中加入 asyncio 模塊，
asyncio.run(result)

### await

#### 範例01

In [ ]:
import asyncio

async def func():
    print("執行協程函數內部代碼")
    # 遇到IO操作掛起當前協程（任務），等IO操作完成之後再繼續往下執行。
    # 當前協程掛起時，事件循環可以去執行其他協程（任務）。
    response = await asyncio.sleep(2)
    print("IO請求結束，結果為：", response)
    
result = func()
asyncio.run(result)

#### 範例02

In [ ]:
import asyncio

async def others():
    print("start")
    await asyncio.sleep(2)
    print('end')
    return '返回值'

async def func():
    print("執行協程函數內部代碼")
    # 遇到 IO 操作掛起當前協程（任務），等 IO 操作完成之後再繼續往下執行。當前協程掛起時，事件循環可以去執行其他協程（任務）。
    response = await others()
    print("IO請求結束，結果為：", response)
    
asyncio.run( func() )

#### 範例03

In [ ]:
import asyncio

async def others():
    print("start")
    await asyncio.sleep(2)
    print('end')
    return '返回值'

async def func():
    print("執行協程函數內部代碼")
    # 遇到IO操作掛起當前協程（任務），等IO操作完成之後再繼續往下執行。當前協程掛起時，事件循環可以去執行其他協程（任務）。
    response1 = await others()
    print("IO請求結束，結果為：", response1)
    response2 = await others()
    print("IO請求結束，結果為：", response2)

asyncio.run( func() )

### Task對象

#### 範例01

In [ ]:
import asyncio

async def func():
    print(1)
    await asyncio.sleep(2)
    print(2)
    return "返回值"

async def main():
    print("main開始")
    # 創建協程，將協程封裝到一個 Task 對象中並立即添加到事件循環的任務列表中，等待事件循環去執行（默認是就緒狀態）。
    task1 = asyncio.create_task(func())
    # 創建協程，將協程封裝到一個Task對象中並立即添加到事件循環的任務列表中，等待事件循環去執行（默認是就緒狀態）。
    task2 = asyncio.create_task(func())
    print("main結束")
    # 當執行某協程遇到 IO 操作時，會自動化切換執行其他任務。
    # 此處的 await 是等待相對應的協程全都執行完畢並獲取結果
    ret1 = await task1
    ret2 = await task2
    print(ret1, ret2)
    
asyncio.run(main())

#### 範例02-最常用

In [ ]:
import asyncio

async def func():
    print(1)
    await asyncio.sleep(2)
    print(2)
    return "返回值"

async def main():
    print("main開始")
    # 創建協程，將協程封裝到 Task 對象中並添加到事件循環的任務列表中，等待事件循環去執行（默認是就緒狀態）。
    # 在調用
    task_list = [
        asyncio.create_task(func(), name="n1"),
        asyncio.create_task(func(), name="n2")
    ]
    print("main結束")
    # 當執行某協程遇到 IO 操作時，會自動化切換執行其他任務。
    # 此處的 await 是等待所有協程執行完畢，並將所有協程的返回值保存到 done
    # 如果設置了 timeout 值，則意味著此處最多等待的秒，完成的協程返回值寫入到 done 中，未完成則寫到 pending 中。
    # 因為 awiat 要接協程對象、task對象等，所以這裡要添加 asyncio.wait() 函式
    done, pending = await asyncio.wait(task_list, timeout=None)
    print(done, pending)
    
asyncio.run(main())

#### 範例03

In [ ]:
import asyncio

async def func():
    print("執行協程函數內部代碼")
    # 遇到IO操作掛起當前協程（任務），等IO操作完成之後再繼續往下執行。當前協程掛起時，事件循環可以去執行其他協程（任務）。
    response = await asyncio.sleep(2)
    print("IO請求結束，結果為：", response)
    
coroutine_list = [func(), func()]
# 錯誤：coroutine_list = [ asyncio.create_task(func()), asyncio.create_task(func()) ]  
# 此處不能直接 asyncio.create_task，因為將 Task 立即加入到事件循環的任務列表，
# 但此時事件循環還未創建，所以會報錯。
# 使用 asyncio.wait 將列表封裝為一個協程，並調用 asyncio.run 實現執行兩個協程
# asyncio.wait 內部會對列表中的每個協程執行 ensure_future，封裝為 Task 對象。
done, pending = asyncio.run( asyncio.wait(coroutine_list) )
print(done, pending)

### asyncio.Future對象

#### 範例01

In [ ]:
import asyncio

async def main():
    # 獲取當前事件循環
    loop = asyncio.get_running_loop()
    # 創建一個任務（Future對象），這個任務什麽都不幹。
    fut = loop.create_future()
    # 等待任務最終結果（Future對象），沒有結果則會一直等下去。
    await fut
    
asyncio.run(main())

#### 範例02

In [ ]:
import asyncio

async def set_after(fut):
    await asyncio.sleep(2)
    fut.set_result("666")
    
async def main():
    # 獲取當前事件循環
    loop = asyncio.get_running_loop()
    # 創建一個任務（Future對象），沒綁定任何行為，則這個任務永遠不知道什麽時候結束。
    fut = loop.create_future()
    # 創建一個任務（Task對象），綁定了 set_after 函數，函數內部在 2s 之後，會給 fut 賦值。
    # 即手動設置 future 任務的最終結果，那麽 fut 就可以結束了。
    await loop.create_task(set_after(fut))
    # 等待 Future 對象獲取 最終結果，否則一直等下去
    data = await fut
    print(data)
    
asyncio.run(main())

### concurrent.futures.Future对象

In [ ]:
import time
from concurrent.futures import Future
from concurrent.futures.thread import ThreadPoolExecutor
from concurrent.futures.process import ProcessPoolExecutor

def func(value):
    time.sleep(1)
    print(value)

pool = ThreadPoolExecutor(max_workers=5)
# 或 pool = ProcessPoolExecutor(max_workers=5)

for i in range(10):
    fut = pool.submit(func, i)
    print(fut)

In [ ]:
import time
import asyncio
import concurrent.futures

def func1():
    # 某個耗時操作
    time.sleep(2)
    return "SB"

async def main():
    loop = asyncio.get_running_loop()
    # 1. Run in the default loop's executor ( 默認 ThreadPoolExecutor )
    # 第一步：內部會先調用 ThreadPoolExecutor 的 submit 方法去線程池中申請一個線程去執行 func1 函數，並返回一個 concurrent.futures.Future 對象
    # 第二步：調用 asyncio.wrap_future 將 concurrent.futures.Future 對象包裝為 asycio.Future 對象。
    # 因為 concurrent.futures.Future 對象不支持 await 語法，所以需要包裝為 asycio.Future 對象才能使用。
    fut = loop.run_in_executor(None, func1)
    result = await fut
    print('default thread pool', result)
    # 2. Run in a custom thread pool:
    # with concurrent.futures.ThreadPoolExecutor() as pool:
    #     result = await loop.run_in_executor(
    #         pool, func1)
    #     print('custom thread pool', result)
    # 3. Run in a custom process pool:
    # with concurrent.futures.ProcessPoolExecutor() as pool:
    #     result = await loop.run_in_executor(
    #         pool, func1)
    #     print('custom process pool', result)

asyncio.run(main())

In [ ]:
import asyncio
import requests

async def download_image(url):
    # 發送網絡請求，下載圖片（遇到網絡下載圖片的IO請求，自動化切換到其他任務）
    print("開始下載:", url)
    loop = asyncio.get_event_loop()
    # requests模塊默認不支持異步操作，所以就使用線程池來配合實現了。
    future = loop.run_in_executor(None, requests.get, url)
    response = await future
    print('下載完成')
    # 圖片保存到本地文件
    file_name = url.rsplit('_')[-1]
    with open(file_name, mode='wb') as file_object:
        file_object.write(response.content)
        
        
if __name__ == '__main__':
    url_list = [
        'https://www3.autoimg.cn/newsdfs/g26/M02/35/A9/120x90_0_autohomecar__ChsEe12AXQ6AOOH_AAFocMs8nzU621.jpg',
        'https://www2.autoimg.cn/newsdfs/g30/M01/3C/E2/120x90_0_autohomecar__ChcCSV2BBICAUntfAADjJFd6800429.jpg',
        'https://www3.autoimg.cn/newsdfs/g26/M0B/3C/65/120x90_0_autohomecar__ChcCP12BFCmAIO83AAGq7vK0sGY193.jpg'
    ]
    tasks = [download_image(url) for url in url_list]
    loop = asyncio.get_event_loop()
    loop.run_until_complete( asyncio.wait(tasks) )

### 異步叠代器

In [ ]:
import asyncio

class Reader(object):
    """ 自定義異步叠代器（同時也是異步可叠代對象） """
    def __init__(self):
        self.count = 0
    async def readline(self):
        # await asyncio.sleep(1)
        self.count += 1
        if self.count == 100:
            return None
        return self.count
    def __aiter__(self):
        return self
    async def __anext__(self):
        val = await self.readline()
        if val == None:
            raise StopAsyncIteration
        return val

async def func():
    # 創建異步可叠代對象
    async_iter = Reader()
    # async for 必須要放在async def函數內，否則語法錯誤。
    async for item in async_iter:
        print(item)
        
asyncio.run(func())

### 異步上下文管理器

In [ ]:
import asyncio

class AsyncContextManager:
    def __init__(self):
        self.conn = None
        
    async def do_something(self):
        # 異步操作數據庫
        return 666
    
    async def __aenter__(self):
        # 異步鏈接數據庫
        self.conn = await asyncio.sleep(1)
        return self
    
    async def __aexit__(self, exc_type, exc, tb):
        # 異步關閉數據庫鏈接
        await asyncio.sleep(1)
        
async def func():
    async with AsyncContextManager() as f:
        result = await f.do_something()
        print(result)
        
asyncio.run(func())

### 小結

中文版：https://docs.python.org/zh-cn/3.8/library/asyncio.html <br>
英文本：https://docs.python.org/3.8/library/asyncio.html

# uvloop

In [ ]:
在項目中想要使用 uvloop 替換 asyncio 的事件循環也非常簡單，只要在代碼中這麽做就行。

In [ ]:
import asyncio
import uvloop

asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())
# 編寫asyncio的代碼，與之前寫的代碼一致。
# 內部的事件循環自動化會變為uvloop
asyncio.run(...)

# 實戰案例

### 異步Redis

#### 範例1：異步操作redis

In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import asyncio
import aioredis

async def execute(address, password):
    print("開始執行", address)
    # 網絡IO操作：創建redis連接
    redis = await aioredis.create_redis(address, password=password)
    # 網絡IO操作：在redis中設置哈希值car，內部在設三個鍵值對，即： redis = { car:{key1:1,key2:2,key3:3}}
    await redis.hmset_dict('car', key1=1, key2=2, key3=3)
    # 網絡IO操作：去redis中獲取值
    result = await redis.hgetall('car', encoding='utf-8')
    print(result)
    redis.close()
    # 網絡IO操作：關閉redis連接
    await redis.wait_closed()
    print("結束", address)
    
asyncio.run(execute('redis://47.93.4.198:6379', "root!2345"))

#### 範例2：連接多個 redis 做操作（遇到 IO 會切換其他任務，提供了性能）

In [ ]:
import asyncio
import aioredis

async def execute(address, password):
    print("開始執行", address)
    # 網絡 IO 操作：先去連接 47.93.4.197:6379，遇到 IO 則自動切換任務，去連接 47.93.4.198:6379
    redis = await aioredis.create_redis_pool(address, password=password)
    # 網絡 IO 操作：遇到 IO 會自動切換任務
    await redis.hmset_dict('car', key1=1, key2=2, key3=3)
    # 網絡 IO 操作：遇到 IO 會自動切換任務
    result = await redis.hgetall('car', encoding='utf-8')
    print(result)
    redis.close()
    # 網絡 IO 操作：遇到 IO 會自動切換任務
    await redis.wait_closed()
    print("結束", address)
    
task_list = [
    execute('redis://47.93.4.197:6379', "root!2345"),
    execute('redis://47.93.4.198:6379', "root!2345")
]
asyncio.run(asyncio.wait(task_list))

更多 redis 操作參考 aioredis 官網：https://aioredis.readthedocs.io/en/v1.3.0/start.html

### 異步MySQL

#### 範例1：

In [ ]:
import asyncio
import aiomysql

async def execute():
    # 網絡IO操作：連接MySQL
    conn = await aiomysql.connect(host='127.0.0.1', port=3306, user='root', password='123', db='mysql', )
    # 網絡IO操作：創建CURSOR
    cur = await conn.cursor()
    # 網絡IO操作：執行SQL
    await cur.execute("SELECT Host,User FROM user")
    # 網絡IO操作：獲取SQL結果
    result = await cur.fetchall()
    print(result)
    # 網絡IO操作：關閉鏈接
    await cur.close()
    conn.close()
    
asyncio.run(execute())

#### 範例02

In [ ]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import asyncio
import aiomysql

async def execute(host, password):
    print("開始", host)
    # 網絡IO操作：先去連接 47.93.40.197，遇到IO則自動切換任務，去連接47.93.40.198:6379
    conn = await aiomysql.connect(host=host, port=3306, user='root', password=password, db='mysql')
    # 網絡IO操作：遇到IO會自動切換任務
    cur = await conn.cursor()
    # 網絡IO操作：遇到IO會自動切換任務
    await cur.execute("SELECT Host,User FROM user")
    # 網絡IO操作：遇到IO會自動切換任務
    result = await cur.fetchall()
    print(result)
    # 網絡IO操作：遇到IO會自動切換任務
    await cur.close()
    conn.close()
    print("結束", host)
    
task_list = [
    execute('47.93.40.197', "root!2345"),
    execute('47.93.40.197', "root!2345")
]

asyncio.run(asyncio.wait(task_list))

### 爬蟲

In [ ]:
import aiohttp
import asyncio

async def fetch(session, url):
    print("發送請求：", url)
    async with session.get(url, verify_ssl=False) as response:
        text = await response.text()
        print("得到結果：", url, len(text))
        
async def main():
    async with aiohttp.ClientSession() as session:
        url_list = [
            'https://python.org',
            'https://www.baidu.com',
            'https://www.pythonav.com'
        ]
        
        tasks = [asyncio.create_task(fetch(session, url)) for url in url_list]
        await asyncio.wait(tasks)
        
if __name__ == '__main__':
    asyncio.run(main())

# 總結

為了提升性能越來越多的框架都在向異步編程靠攏，例如：sanic、tornado、django3.0、django channels 組件 等，用更少資源可以做處理更多的事，何樂而不為呢。